This is the Training Stage Implementation of [Spoonful Bangkit 2024 Casptone Project](https://github.com/Spoonful-Capstone/Spoonful-ML/tree/master). Go to [this link](https://colab.research.google.com/drive/17vwft3NtaE2tfWTiHliZxBnqm7EDBV-R?usp=sharing#scrollTo=P45xPDz_DcqS) if you want to open it on colab

## Install and Import Dependencies

In [ ]:
# @title <p> Install dependencies
!pip install roboflow --quiet &> /dev/null

In [ ]:
# @title <p>Essential Import
import os, shutil, json
from PIL import Image
from zipfile import ZipFile
import matplotlib.pyplot as plt
import numpy as np, pandas as pd, random as rd
import warnings
import pathlib
from collections import Counter
warnings.filterwarnings("ignore")

In [ ]:
# @title <p> Import Modelling Dependencies
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import MobileNet, MobileNetV3Small, EfficientNetB0, EfficientNetB7

from roboflow import Roboflow
from sklearn.metrics import classification_report, accuracy_score

## Get Dataset

### Download Datasets

In [ ]:
# @title <p> Make a Helper Function
def relocate_dataset(INITIAL_PATH, DATASET_PATH, TRAIN_SIZE, VAL_SIZE):
  os.makedirs(DATASET_PATH, exist_ok=True)

  for mode in ['train', 'val', 'test']:

    os.makedirs(os.path.join(DATASET_PATH, mode), exist_ok=True)
    for food in os.listdir(INITIAL_PATH):
      os.makedirs(os.path.join(DATASET_PATH, mode, food), exist_ok=True)

  for food in os.listdir(INITIAL_PATH):

    for i, img in enumerate(os.listdir(os.path.join(INITIAL_PATH, food))):

      count_food_img = len(os.listdir(os.path.join(INITIAL_PATH, food)))

      if i <= int(TRAIN_SIZE * count_food_img) :
        shutil.copy(os.path.join(INITIAL_PATH, food, img), os.path.join(DATASET_PATH, 'train', food, img))

      elif i <= int((TRAIN_SIZE + VAL_SIZE) * count_food_img):
        shutil.copy(os.path.join(INITIAL_PATH, food, img), os.path.join(DATASET_PATH, 'val', food, img))

      elif i > int((TRAIN_SIZE + VAL_SIZE) * count_food_img):
        shutil.copy(os.path.join(INITIAL_PATH, food, img), os.path.join(DATASET_PATH, 'test', food, img))

  shutil.rmtree(INITIAL_PATH)

# @title <p> Print food len
def print_dataset_len(DATASET_PATH):
  for food in os.listdir(os.path.join(DATASET_PATH, 'train')):

    train_food_path = os.listdir(os.path.join(DATASET_PATH, 'train', food))
    val_food_path = os.listdir(os.path.join(DATASET_PATH, 'val', food))
    test_food_path = os.listdir(os.path.join(DATASET_PATH, 'test', food))

    print(f"{food} : ")
    print(f"Train : {len(train_food_path)} | Val : {len(val_food_path)} | Test : {len(test_food_path)}")


def import_kaggle_json():
  from google.colab import files
  files.upload()

  ! mkdir ~/.kaggle
  ! cp kaggle.json ~/.kaggle
  ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# @title <p>Choose Datasets
# Kaggle Food Image Classification https://www.kaggle.com/datasets/harishkumardatalab/food-image-classification-dataset
Kaggle_Dataset   = False        #@param {type:"boolean"}
# Roboflow Indonesian Food Image Classification https://universe.roboflow.com/bangkit/indonesian-food-pedsx
Roboflow_Dataset = True         #@param {type:"boolean"}

if os.path.isdir('dataset'):
  !rm -r 'dataset'
if os.path.isdir('Food Classification dataset'):
  !rm -r 'Food Classification dataset'
if os.path.isdir('Indonesian-Food-1'):
  !rm -r 'Indonesian-Food-1'

if Kaggle_Dataset :
  if not os.path.isfile('kaggle.json'):
    import_kaggle_json()
  !kaggle datasets download 'harishkumardatalab/food-image-classification-dataset'
  !unzip '/content/food-image-classification-dataset.zip' &> /dev/null
  !rm '/content/food-image-classification-dataset.zip'

  DATASET_PATH = 'dataset'

  TRAIN_SIZE = 0.8
  VAL_SIZE = 0.1
  relocate_dataset('Food Classification dataset', DATASET_PATH, TRAIN_SIZE, VAL_SIZE)

if Roboflow_Dataset :

  rf = Roboflow(api_key="1UnUQCCfSuu44HS6CrHe")
  project = rf.workspace("bangkit").project("indonesian-food-pedsx")
  version = project.version(1)
  dataset = version.download("folder")

  os.rename('Indonesian-Food-1/valid', 'Indonesian-Food-1/val')
  DATASET_PATH = 'Indonesian-Food-1'

print_dataset_len(DATASET_PATH)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Indonesian-Food-1 in folder:: 100%|██████████| 962/962 [00:00<00:00, 8954.13it/s]

16. Soto Banjar : 
Train : 33 | Val : 8 | Test : 3
14. Sate Madura : 
Train : 34 | Val : 12 | Test : 9
12. Rawon : 
Train : 40 | Val : 10 | Test : 5
04. Gudeg : 
Train : 39 | Val : 10 | Test : 5
08. Nasi Pecel : 
Train : 35 | Val : 8 | Test : 7
01. Ayam Betutu : 
Train : 37 | Val : 10 | Test : 8
07. Nasi Kuning : 
Train : 23 | Val : 11 | Test : 4
05. Kerak Telor : 
Train : 36 | Val : 6 | Test : 3
03. Coto Makassar : 
Train : 35 | Val : 11 | Test : 3
09. Papeda : 
Train : 41 | Val : 11 | Test : 3
15. Serabi : 
Train : 39 | Val : 9 | Test : 7
11. Peuyeum : 
Train : 37 | Val : 16 | Test : 2
18. Tahu Sumedang : 
Train : 40 | Val : 5 | Test : 3
02. Beberuk Terong : 
Train : 26 | Val : 10 | Test : 4
06. Mie Aceh : 
Train : 28 | Val : 8 | Test : 4
10. Pempek : 
Train : 37 | Val : 14 | Test : 4
17. Soto Lamongan : 
Train : 39 | Val : 10 | Test : 6
13. Rendang : 
Train : 36 | Val : 10 | Test : 9


## Preprocessing

In [ ]:
# @title <p> Make a Function to Create Generator
def create_generator(train_path,
                     val_path,
                     test_path,
                     BATCH_SIZE = 32,
                     IMG_SHAPE=256,
                     class_mode='categorical',
                     ):

  train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range = 0.1,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip=True,
    fill_mode='nearest'
  )

  test_len = sum([len(os.listdir(os.path.join(TEST_PATH, label))) for label in os.listdir(TEST_PATH)])

  test_datagen = ImageDataGenerator(rescale = 1.0/255.)

  train_generator = train_datagen.flow_from_directory(train_path, batch_size=BATCH_SIZE,
                                                     shuffle=True, class_mode=class_mode,
                                                     target_size=(IMG_SHAPE, IMG_SHAPE))
  val_generator = test_datagen.flow_from_directory(val_path, batch_size=BATCH_SIZE,
                                                     shuffle=False, class_mode=class_mode,
                                                     target_size=(IMG_SHAPE, IMG_SHAPE))
  test_generator = test_datagen.flow_from_directory(test_path, batch_size=test_len,
                                                     shuffle=False, class_mode=class_mode,
                                                     target_size=(IMG_SHAPE, IMG_SHAPE))

  return train_generator, val_generator, test_generator

In [ ]:
# @title <p> Make a dataset object
BATCH_SIZE = 32 #@param{type:"integer"}
IMG_SHAPE = 256 #@param{type:"integer"}

COUNT_LABEL = len(os.listdir(os.path.join(DATASET_PATH, 'train')))

TRAIN_PATH = os.path.join(DATASET_PATH, 'train')
VAL_PATH = os.path.join(DATASET_PATH, 'val')
TEST_PATH = os.path.join(DATASET_PATH, 'test')

train_generator, val_generator, test_generator = create_generator(TRAIN_PATH, VAL_PATH, TEST_PATH, BATCH_SIZE, IMG_SHAPE)

Found 635 images belonging to 18 classes.
Found 179 images belonging to 18 classes.
Found 89 images belonging to 18 classes.


## Modelling

### Model Building

In [ ]:
# @title <p> Finetune the Inception Model Functoin
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.

dense_layer = 128 #@param {type:'raw'}


def inception_model(IMG_SHAPE, COUNT_LABEL):
  !wget --no-check-certificate \
      https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
      -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

  local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
  pre_trained_model = InceptionV3(input_shape = (IMG_SHAPE, IMG_SHAPE, 3),
                                  include_top = False,
                                  weights = None)

  pre_trained_model.load_weights(local_weights_file)
  for layer in pre_trained_model.layers:
    layer.trainable = False

  # Choose `mixed7` as the last layer of your base model
  last_layer = pre_trained_model.get_layer('mixed7')
  print('last layer output shape: ', last_layer.output_shape)
  last_output = last_layer.output

  x = layers.Flatten()(last_output)
  x = layers.Dense(dense_layer, activation='relu')(x)
  x = layers.Dropout(0.2)(x)
  x = layers.Dense  (COUNT_LABEL, activation='softmax')(x)

  model = Model(pre_trained_model.input, x)
  return model

In [ ]:
# @title <p> Finetune the MobileNet Model Functoin
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.

# dense_layer = 1024 #@param {type:'raw'}

def mobilenet_model(IMG_SHAPE, COUNT_LABEL):
  # !wget https://github.com/fchollet/deep-learning-models/releases/download/v0.6/mobilenet_1_0_224_tf_no_top.h5
  local_weights_file = 'mobilenet_1_0_224_tf_no_top.h5'
  pre_trained_model = MobileNet(include_top=False, input_shape=(IMG_SHAPE, IMG_SHAPE, 3))

  pre_trained_model.load_weights(local_weights_file)
  for layer in pre_trained_model.layers:
    layer.trainable = False

  last_layer = pre_trained_model.get_layer('conv_pw_13_bn')
  print('last layer output shape: ', last_layer.output_shape)
  last_output = last_layer.output

  x = layers.Flatten()(last_output)
  x = layers.Dense(dense_layer, activation='relu')(x)
  x = layers.Dropout(0.2)(x)
  x = layers.Dense  (COUNT_LABEL, activation='softmax')(x)

  model = Model(pre_trained_model.input, x)
  print(model.summary())
  return model

In [ ]:
# @title <p> Finetune the EfficientNet Model Functoin
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.

dense_layer = 256 #@param {type:'raw'}

def efficientnet_model(IMG_SHAPE, COUNT_LABEL):
  # !wget --no-check-certificate \
  #     https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
  #     -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

  # local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
  pre_trained_model = EfficientNetB7 (input_shape = (IMG_SHAPE, IMG_SHAPE, 3),
                                  include_top = False,
                                  weights = 'imagenet')

  # pre_trained_model.load_weights(local_weights_file)
  for layer in pre_trained_model.layers:
    layer.trainable = False

  last_layer = pre_trained_model.get_layer('top_bn')
  print('last layer output shape: ', last_layer.output_shape)
  last_output = last_layer.output

  x = layers.Flatten()(last_output)
  x = layers.Dense(dense_layer, activation='relu')(x)
  x = layers.Dropout(0.2)(x)
  x = layers.Dense  (COUNT_LABEL, activation='softmax')(x)

  model = Model(pre_trained_model.input, x)
  print(model.summary())
  return model

In [ ]:
# @title <p> Make a Custom Model Function

conv_layer = [16, 32, 64] #@param {type:'raw'}
dense_layer_custom = [1024, 32] #@param {type:'raw'}
batch_norm = False #@param {type:'raw'}
dropout = 0.2 #@param {type:'raw'}

def create_costum_model(IMG_SHAPE, COUNT_LABEL):
  class ConvBlock(tf.keras.Model):
    def __init__(self, units, kernel_size=(3,3), padding=None, batch_norm=True):
      super(ConvBlock, self).__init__()
      self.conv = tf.keras.layers.Conv2D(units, kernel_size, padding=padding, activation='relu')
      if batch_norm :
        self.bn = tf.keras.layers.BatchNormalization()
      self.maxpool = tf.keras.layers.MaxPool2D((2,2))

    def call(self, inputs):
      x = self.conv(inputs)
      if batch_norm :
        x = self.bn(x)

      return self.maxpool(x)

  class DenseBlock(tf.keras.Model):
    def __init__(self, units, dropout=0.2):
      super(DenseBlock, self).__init__()
      self.linear = tf.keras.layers.Dense(units, activation='relu')
      self.dropout = tf.keras.layers.Dropout(0.2)
      self.ln = tf.keras.layers.Normalization()

    def call(self, inputs):
      x = self.linear(inputs)
      x = self.dropout(x)

      return self.ln(x)

  class CustomModel(tf.keras.Model):
    def __init__(self):
      super().__init__()
      self.conv = {f'convblock{i + 1}' : ConvBlock(filters, (3,3), padding='same') for i, filters in enumerate(conv_layer)}

      self.flatten = tf.keras.layers.Flatten()
      self.dense = {f'denseblock{i + 1}' : DenseBlock(units) for i, units in enumerate(dense_layer_custom)}

      self.out = tf.keras.layers.Dense(COUNT_LABEL, activation='softmax')


    def call(self, x):

      for i in range(1, len(self.conv) + 1) :
        x = self.conv[f'convblock{i}'](x)
      x = self.flatten(x)
      for i in range(1, len(self.dense) + 1) :
        x = self.dense[f'denseblock{i}'](x)
      x = self.out(x)
      return x

  model = CustomModel()
  model.build((None, IMG_SHAPE, IMG_SHAPE, 3))
  return model

In [ ]:
# @title <p> Make a Model
inception = True #@param {type:'boolean'}
mobilenet = False #@param {type:'boolean'}
efficientnet = False #@param {type:'boolean'}
costum = False #@param {type:'boolean'}

if inception :
  model = inception_model(IMG_SHAPE, COUNT_LABEL)
if mobilenet :
  model = mobilenet_model(IMG_SHAPE, COUNT_LABEL)
if efficientnet :
  model = efficientnet_model(IMG_SHAPE, COUNT_LABEL)
if costum :
  model = create_costum_model(IMG_SHAPE, COUNT_LABEL)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

--2024-06-04 08:57:10--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.207, 74.125.197.207, 74.125.135.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   295MB/s    in 0.3s    

2024-06-04 08:57:11 (295 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

last layer output shape:  (None, 14, 14, 768)
Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_22 (InputLayer)       [(No

### Training

In [ ]:
# @title <p> Make a Training Function
def train_model(model,
                train_generator, val_generator,
                EPOCHS, LEARNING_RATE, checkpoint_path) :
  # Compile Model
  model.compile(optimizer = Adam(learning_rate=LEARNING_RATE),
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])

  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
  # Train Model
  history = model.fit(
              train_generator,
              validation_data = val_generator,
              steps_per_epoch = len(train_generator),
              epochs = EPOCHS,
              validation_steps =  len(val_generator),
              verbose = 2,
              callbacks=[cp_callback])

  return history

In [ ]:
# @title <p> Train Model
EPOCHS = 40  #@param {type:'integer'}
LEARNING_RATE = 1e-3  #@param {type:'raw'}
checkpoint_path = "training_cp_efficient/cp.ckpt" #@param {type:'raw'}

result = train_model(model,
                     train_generator,
                     val_generator, EPOCHS,
                     LEARNING_RATE, checkpoint_path )

Epoch 1/40

Epoch 1: saving model to training_cp_efficient/cp.ckpt
20/20 - 21s - loss: 4.2407 - accuracy: 0.1307 - val_loss: 2.4532 - val_accuracy: 0.2682 - 21s/epoch - 1s/step
Epoch 2/40

Epoch 2: saving model to training_cp_efficient/cp.ckpt
20/20 - 17s - loss: 2.4601 - accuracy: 0.2299 - val_loss: 2.0816 - val_accuracy: 0.3240 - 17s/epoch - 850ms/step
Epoch 3/40

Epoch 3: saving model to training_cp_efficient/cp.ckpt
20/20 - 18s - loss: 2.0625 - accuracy: 0.3528 - val_loss: 1.7787 - val_accuracy: 0.4860 - 18s/epoch - 924ms/step
Epoch 4/40

Epoch 4: saving model to training_cp_efficient/cp.ckpt
20/20 - 18s - loss: 1.7062 - accuracy: 0.4646 - val_loss: 1.5604 - val_accuracy: 0.5140 - 18s/epoch - 913ms/step
Epoch 5/40

Epoch 5: saving model to training_cp_efficient/cp.ckpt
20/20 - 14s - loss: 1.4794 - accuracy: 0.5276 - val_loss: 1.3014 - val_accuracy: 0.6257 - 14s/epoch - 695ms/step
Epoch 6/40

Epoch 6: saving model to training_cp_efficient/cp.ckpt
20/20 - 43s - loss: 1.3389 - accurac

KeyboardInterrupt: 

### Export

In [ ]:
# @title <p> Export Model Function
def export_model_lite(model, export_path, quantized=True):
  tf.saved_model.save(model, export_path)

  converter = tf.lite.TFLiteConverter.from_saved_model(export_path)
  tflite_model = converter.convert()

  tflite_model_file = pathlib.Path(f'{export_path}.tflite')
  tflite_model_file.write_bytes(tflite_model)

  if quantized :
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]

    tflite_models_fp16_file = pathlib.Path(f'{export_path}-quantized.tflite')

    tflite_fp16_model = converter.convert()
    tflite_models_fp16_file.write_bytes(tflite_fp16_model)

In [ ]:
# @title <p> Export Model
export_path = 'inception-128' #@param {type:'raw'}
export_model_lite(model, export_path, quantized=True)

## Evaluation

In [ ]:
# @title <p> Plot Loss
plt.plot(result.history['accuracy'])
plt.plot(result.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'result' is not defined

In [ ]:
# @title <p> Plot Accuracy
plt.plot(result.history['accuracy'])
plt.plot(result.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# @title <p> Evaluation with Test Data
y_pred = tf.math.argmax(model(test_generator[0][0]), axis=1)
y_true = tf.math.argmax(test_generator[0][1], axis=1)
target_names = test_generator.class_indices.keys()

eval = classification_report(y_true, y_pred, target_names = target_names)
print(eval)

                    precision    recall  f1-score   support

   01. Ayam Betutu       0.89      1.00      0.94         8
02. Beberuk Terong       1.00      1.00      1.00         4
 03. Coto Makassar       0.60      1.00      0.75         3
         04. Gudeg       0.80      0.80      0.80         5
   05. Kerak Telor       1.00      1.00      1.00         3
      06. Mie Aceh       1.00      1.00      1.00         4
   07. Nasi Kuning       1.00      1.00      1.00         4
    08. Nasi Pecel       1.00      0.86      0.92         7
        09. Papeda       0.75      1.00      0.86         3
        10. Pempek       0.75      0.75      0.75         4
       11. Peuyeum       1.00      1.00      1.00         2
         12. Rawon       1.00      1.00      1.00         5
       13. Rendang       1.00      0.89      0.94         9
   14. Sate Madura       1.00      1.00      1.00         9
        15. Serabi       1.00      0.86      0.92         7
   16. Soto Banjar       0.50      0.67